In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from gcn import GCN  # Import GCN model
from pytorch_metattack import Metattack  # Import Metattack for evasion attack
from utils import load_data, normalize_adj_tensor, accuracy
import scipy.sparse as sp


In [2]:
#Loading Cora (using cora.npz)
dataset = "cora"  # Change to "citeseer" or "pubmed" if needed
adj, features, labels = load_data(dataset)


Loading cora dataset...
reading cora...
Selecting 1 largest connected components


In [3]:
# Convert adjacency matrix to PyTorch tensor and move to correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

adj = torch.FloatTensor(adj.todense()).to(device)  # ✅ Convert to PyTorch tensor and move to GPU if available
adj = normalize_adj_tensor(adj)  # ✅ Now normalize it

features = torch.FloatTensor(features.todense()).to(device)  # ✅ Move features to device
labels = torch.LongTensor(labels).to(device)

In [4]:
# Define train/test split (using 60% train, 20% validation, 20% test)
idx_train = torch.arange(int(0.6 * len(labels)), device=device)
idx_val = torch.arange(int(0.6 * len(labels)), int(0.8 * len(labels)), device=device)
idx_test = torch.arange(int(0.8 * len(labels)), len(labels), device=device)


In [5]:
# Training GCN on Clean graph
gcn = GCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5).to(device)
optimizer = optim.Adam(gcn.parameters(), lr=0.01, weight_decay=5e-4)

In [6]:
# Training Loop
print("\nTraining GCN on Clean Graph...")
for epoch in range(200):
    gcn.train()
    optimizer.zero_grad()
    output = gcn(features, adj)
    loss = F.nll_loss(output[idx_train], labels[idx_train])
    loss.backward()
    optimizer.step()


Training GCN on Clean Graph...


In [10]:
# Evaluate on clean graph
gcn.eval()
output = gcn(features, adj)
clean_preds = output.argmax(dim=1)
clean_acc = accuracy(output[idx_test], labels[idx_test])

print(f"Clean Test Accuracy: {clean_acc.item() * 100:.2f}%")


Clean Test Accuracy: 81.10%


In [13]:
print("\n⚠️  Running Metattack (Evasion)...")

attack_model = Metattack(nfeat=features.shape[1], hidden_sizes=[16], nclass=labels.max().item() + 1,
                         nnodes=adj.shape[0], dropout=0.5, train_iters=0, attack_features=False, device=device).to(device)

# Define the number of perturbations (5% of edges)
perturbations = int(0.05 * adj.sum())
perturbed_adj = attack_model(features, adj, labels, idx_train, idx_test, perturbations)


⚠️  Running Metattack (Evasion)...
=== training surrogate model to predict unlabled data for self-training


Perturbing graph:   0%|                                                                         | 0/114 [00:00<?, ?it/s]

GCN loss on unlabled data: 2.0583696365356445
GCN acc on unlabled data: 0.09859154929577466
attack loss: 2.0508623123168945


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.